#### Spotlight

The Quickstart Example

In [1]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [2]:
dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

In [3]:
train

<Interactions dataset (944 users x 1683 items x 80000 interactions)>

Not helpful... need to go deeper:

In [4]:
train.user_ids

array([452, 343, 314, ..., 661, 407,  85], dtype=int32)

In [5]:
train.item_ids

array([156, 286,  90, ..., 298, 493, 520], dtype=int32)

In [6]:
train.ratings

array([4., 4., 2., ..., 3., 3., 3.], dtype=float32)

And peek inside:

In [7]:
train.tocsr().todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 5., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 5., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

The model:

In [8]:
model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

In [9]:
mrr[:10]

array([0.02688494, 0.04699203, 0.01882516, 0.06621178, 0.04774302,
       0.02652233, 0.01561176, 0.09638607, 0.00632509, 0.04397489])

### GitHub Stars

Data retrived from scraping GitHub:

In [10]:
import pandas as pd 

df = pd.read_csv('data/stars.csv')

In [11]:
df.sample(5)

,user,repo,description,language,stargazers
18771,gzliuzhijun,iver56/audiomentations,A Python library for audio data augmentation. ...,Python,87
16099,azraelmt,eosterberg/terminaljs,terminal.js is a dead simple JavaScript librar...,JavaScript,192
10644,chendamon,maxhumber/gif,✨ Better animated Matplotlib gifs,Python,157
26463,javad94,pytest-dev/pytest,The pytest framework makes it easy to write sm...,Python,5596
38506,anfederico,gersteinlab/idash19bc,Solution to iDASH'19 challenge track 1,JavaScript,2


In [12]:
df['language'].value_counts()

Python              25702
JavaScript           6459
Jupyter Notebook     3777
Go                   2571
C++                  2544
                    ...  
Isabelle                1
VCL                     1
Thrift                  1
ZIL                     1
AngelScript             1
Name: language, Length: 178, dtype: int64

In [13]:
df = df[df.language == 'Python']
df = df[~df['repo'].isin(['maxhumber/gif', 'maxhumber/gazpacho'])]

In [14]:
df.shape

(25380, 5)

In [15]:
len(df['repo'].unique())

12222

In [16]:
len(df['user'].unique())

326

In [17]:
df.head(3)

,user,repo,description,language,stargazers
0,sbarman-mi9,as-ideas/ForwardTacotron,⏩ Generating speech in a single forward pass w...,Python,97
1,sbarman-mi9,abhishekkrthakur/bert-sentiment,NaN,Python,21
4,sbarman-mi9,EmilyAlsentzer/clinicalBERT,repository for Publicly Available Clinical BER...,Python,160


In [18]:
from spotlight.interactions import Interactions

In [19]:
# won't work
interactions = Interactions(df['user'], df['repo'])

TypeError: must be str, not int

"Everything must be a number"

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

In [22]:
users = user_encoder.fit_transform(df['user'])
items = item_encoder.fit_transform(df['repo'])

In [23]:
interactions = Interactions(users, items)

In [24]:
interactions

<Interactions dataset (326 users x 12222 items x 25380 interactions)>

Be a good Data Scientist:

In [25]:
import numpy as np
from spotlight.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(42))

In [26]:
model = ImplicitFactorizationModel(loss='pointwise', n_iter=20)

In [27]:
model.fit(train)

In [28]:
df.head(3)

,user,repo,description,language,stargazers
0,sbarman-mi9,as-ideas/ForwardTacotron,⏩ Generating speech in a single forward pass w...,Python,97
1,sbarman-mi9,abhishekkrthakur/bert-sentiment,NaN,Python,21
4,sbarman-mi9,EmilyAlsentzer/clinicalBERT,repository for Publicly Available Clinical BER...,Python,160


Examining one user:

In [29]:
garrry = df[df['user'] == 'garrrychan']

In [30]:
garrry['repo'].values

array(['mnielsen/neural-networks-and-deep-learning',
       'brendan-rius/jupyter-c-kernel', 'google-research/uda',
       'hhatto/autopep8', 'MaxHalford/prince',
       'scikit-learn-contrib/sklearn-pandas', 'modin-project/modin',
       'VikParuchuri/apartment-finder', 'uber-research/parallax',
       'prabhupant/python-ds', 'tensorflow/nmt',
       'huggingface/transformers', 'h5py/h5py', 'google-research/bert',
       'sloria/TextBlob', 'openai/gpt-2', 'matsui528/rii',
       'ResidentMario/missingno', 'lmcinnes/enstop',
       'phatpiglet/autocorrect', 'barrust/pyspellchecker',
       'seatgeek/fuzzywuzzy', 'maxhumber/chart', 'apache/airflow',
       'graphql-python/graphene', 'maxhumber/marc', 'spotify/luigi',
       'garrrychan/recipe_recommender_system',
       'CamDavidsonPilon/lifetimes', 'maciejkula/spotlight',
       'lyst/lightfm', 'practical-recommender-systems/moviegeek',
       'uwescience/TrafficCruising-DSSG2017'], dtype=object)

Looking at the user_id:

In [31]:
u = user_encoder.transform(['garrrychan'])

And all of the items:

In [32]:
item_encoder.classes_

array(['00111000/Imports-in-Python', '05bit/peewee-async',
       '0Kee-Team/WatchAD', ..., 'zzw922cn/Automatic_Speech_Recognition',
       'zzzDavid/ICDAR-2019-SROIE', 'zzzeek/sqlalchemy'], dtype=object)

In [33]:
preds = model.predict(u, np.arange(len(item_encoder.classes_)))

In [34]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': preds
}).sort_values('pred', ascending=False)

,repo,pred
11555,vibora-io/vibora,11.118851
8760,openai/gpt-2,10.968850
11142,tiangolo/fastapi,9.924666
413,CorentinJ/Real-Time-Voice-Cloning,9.659683
8148,modin-project/modin,9.625760
...,...,...
1008,JustDoPython/python-100-day,-15.278611
5506,gnemoug/distribute_crawler,-15.671831
582,ElsevierDev/elsapy,-15.810594
11710,wb14123/seq2seq-couplet,-16.038507


Evaulating the model:

In [35]:
from spotlight.evaluation import precision_recall_score

In [36]:
precision, recall = precision_recall_score(model, test, train, k=10)

In [37]:
precision.mean()

0.028104575163398697

In [38]:
recall.mean()

0.022994655493174938

Serialize:

In [39]:
import torch 

torch.save(model, 'model.spot')

In [40]:
del model

In [41]:
model = torch.load('model.spot')

Predict on another random user:

In [42]:
u = user_encoder.transform(['RandomOS'])

In [43]:
model.predict(u, np.arange(len(item_encoder.classes_)))

array([-7.4168816, -4.8445168, -4.4747944, ..., -5.180284 , -8.987839 ,
       -6.461801 ], dtype=float32)

In [44]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': model.predict(u, np.arange(len(item_encoder.classes_)))
}).sort_values('pred', ascending=False).head(20)

,repo,pred
9426,python-poetry/poetry,16.439638
11142,tiangolo/fastapi,14.439381
1547,PostHog/posthog,12.498369
9566,r1chardj0n3s/parse,12.357760
11783,willmcgugan/rich,11.518423
6281,jakubroztocil/httpie,11.313591
10636,spotify/luigi,11.249058
1256,Miserlou/Zappa,10.943411
496,Delgan/loguru,10.470609
10370,sherlock-project/sherlock,10.465134


Actual likes:

In [45]:
df[df['user'] == 'RandomOS']['repo']

58885               tortoise/tortoise-orm
58887                 mingrammer/diagrams
58905                       spulec/uncurl
58913         mozilla-iot/webthing-python
58922                       spotify/luigi
58924                    python-trio/trio
58925                       ranger/ranger
58928              googlefonts/noto-emoji
58937                Synss/python-mbedtls
58940                    pipxproject/pipx
58941                       linkedin/shiv
58942                      pantsbuild/pex
58946        PythonCharmers/python-future
58947     alan-turing-institute/CleverCSV
58948                 emeryberger/scalene
58949                 aouinizied/nfstream
58950               ionelmc/python-hunter
58951                      wolever/pip2pi
58956          fossasia/open-event-server
58994                python-poetry/poetry
58997                     thumbor/thumbor
59004                       holoviz/panel
59019             0xInfection/Awesome-WAF
59020    swisskyrepo/PayloadsAllTh

### But what if the user is brand new?~

In [46]:
user_encoder.transform(['maxhumber'])

ValueError: y contains previously unseen labels: ['maxhumber']

In [47]:
model.predict(1993, np.arange(len(item_encoder.classes_)))

ValueError: Maximum user id greater than number of users in model.